# Requirements

## Environment

1. Anaconda Python distribution (tested with _Miniconda3-py39\_4.12.0-Linux-x86\_64.sh_, _conda v. 4.12.0_).
2. Jupyter server (see _extras/jupyter\_server.sh_ for details).
3. Anaconda environments (run _setup\_conda\_envs.sh_).
4. gmsh (not necessary if you already have meshes in either MSH or XDMF format).


## Setup

### Mesh

You need to have a mesh in XDMF format.  Try:

    conda activate kesi37
    cd extras/FEM/meshes
    snakemake meshes/single_sphere_composite/finest.xdmf -j 1
    
It may take a while.

Now you have the following files created:
- _meshes/single\_sphere\_composite/finest.xdmf_,
- _meshes/single\_sphere\_composite/finest.h5_,
- _meshes/single\_sphere\_composite/finest\_boundaries.xdmf_,
- _meshes/single\_sphere\_composite/finest\_boundaries.h5_,
- _meshes/single\_sphere\_composite/finest\_subdomains.xdmf_,
- _meshes/single\_sphere\_composite/finest\_subdomains.h5_

(_\*.xdmf_ files are headers for _\*.h5_ files).  The files are derived from _meshes/single\_sphere\_composite/finest.msh_ mesh saved in _gmsh_ format, which is created from blueprint in the _meshes/single\_sphere\_composite/finest.geo_.  The blueprint itself is derived from a template _single\_sphere\_composite.geo.template_.  

The template may also be used to derive meshes of other resolutions:
- _meshes/single\_sphere\_composite/finer.xdmf_,
- _meshes/single\_sphere\_composite/fine.xdmf_,
- _meshes/single\_sphere\_composite/normal.xdmf_,
- _meshes/single\_sphere\_composite/coarse.xdmf_,
- _meshes/single\_sphere\_composite/coarser.xdmf_, and
- _meshes/single\_sphere\_composite/coarsest.xdmf_.

For single sphere geometry there are also other templates available:
- _single\_sphere\_plain.geo.template_ (with simpler mesh structure), and
- _single\_sphere\_uniform\_cortex.geo.template_ (with uniform element length in the cortical volume).

<!-- 
> The recommended meshes to be derived are:
> - _single\_sphere\_composite\_finest.xdmf_,
> - _single\_sphere\_plain\_finer.xdmf_, 
> - _single\_sphere\_plain\_finest.xdmf_, 
> - _single\_sphere\_uniform\_cortex\_fine.xdmf_,
> - _single\_sphere\_uniform\_cortex\_finer.xdmf_, and
> - _single\_sphere\_uniform\_cortex\_finest.xdmf_.
-->

### Model properties

For every mesh additional information is necessary, like conductivity of its compartments.  Such information is stored in the following files (appropriate for the tutorial marked in bold) in the _extras/FEM/model\_properties_ directory:
- _circular\_slice.ini_,
- **_single\_sphere.ini_**,
- _four\_spheres\_csf\_1\_mm.ini_,
- _four\_spheres\_csf\_1\_mm\_separate\_cortex.ini_, and
- _four\_spheres\_csf\_3\_mm.ini_.


Format of such file is:

    [<compartment name>]
    volume = <volume number>
    conductivity = <conductivity in SI units>
    
for a compartment and:

    [<surface name>]
    surface = <surface number>
    
for a boundary.  Additional information may be provided, like radius, thickness, or conductivity associated with external surface for subtraction method.

### electrodes

In the _extras/FEM/electrode\_locations_ you can find examplary positions of electrodes in appropriate sections of INI files:

    [<electrode name>]
    x = <X coordinate in meters>
    y = <Y coordinate in meters>
    z = <Z coordinate in meters>

# Preprocessing

Lets define positions of three point electrodes::

    [first]
    x = 0
    y = 0
    z = 0.079
    
    [second]
    x = 0.01
    y = 0
    z = 0.07
    
    [third]
    x = 0.01
    y = 0.01
    z = 0.07
    
Write the positions as _extras/FEM/electrode\_locations/tutorial/single\_sphere.ini_.
 
For every electrode solve for the leadfield correction (at least 5.9 GB of free RAM is required):

    cd extras
    python paper_solve_sphere_on_plate.py \
      --mesh FEM/meshes/meshes/single_sphere_composite/finest.xdmf \
      --degree 3 \
      --config FEM/model_properties/single_sphere.ini \
      --grounded-plate-edge-z -0.088 \
      --electrodes FEM/electrode_locations/tutorial/single_sphere.ini \
      --name first \
      --output FEM/solutions/tutorial/single_sphere/first.ini
    python paper_solve_sphere_on_plate.py \
      --mesh FEM/meshes/meshes/single_sphere_composite/finest.xdmf \
      --degree 3 \
      --config FEM/model_properties/single_sphere.ini \
      --grounded-plate-edge-z -0.088 \
      --electrodes FEM/electrode_locations/tutorial/single_sphere.ini \
      --name second \
      --output FEM/solutions/tutorial/single_sphere/second.ini
    python paper_solve_sphere_on_plate.py \
      --mesh FEM/meshes/meshes/single_sphere_composite/finest.xdmf \
      --degree 3 \
      --config FEM/model_properties/single_sphere.ini \
      --grounded-plate-edge-z -0.088 \
      --electrodes FEM/electrode_locations/tutorial/single_sphere.ini \
      --name third \
      --output FEM/solutions/tutorial/single_sphere/third.ini

> Note, that for spherical geometries a dedicated tool
> `paper_solve_sphere_on_plate.py` is used, with an
> additional parameter `--grounded-plate-edge-z`.

| Parameter  | Description  |
|:------------|:--------------|
| `--mesh`   | FEM mesh |
| `--degree` | degree of FEM elements |
| `--config` | physical model configuration (conductivity of subdomains etc.) |
| `--grounded-plate-edge-z` | Z-coordinate of the edge of the grounded plate |
| `--electrodes` | definition of electrode positions |
| `--name` | name of the electrode for which leadfield correction is to be calculated |
| `--output` | _\*.ini_ file for solution metadata (path to the file with the calculated function is the same up to the _\*.h5_ extension) |

Sample the solution on NxNxN grid, where `N = 2**K + 1`:

    mkdir -p FEM/solutions/tutorial/single_sphere/sampled/9/
    python paper_sample_spherical_solution.py \
      -k 9 \
      --fill 0 \
      --sampling-radius 0.090 \
      --config FEM/solutions/tutorial/single_sphere/first.ini \
      --output FEM/solutions/tutorial/single_sphere/sampled/9/first.npz
    python paper_sample_spherical_solution.py \
      -k 9 \
      --fill 0 \
      --sampling-radius 0.090 \
      --config FEM/solutions/tutorial/single_sphere/second.ini \
      --output FEM/solutions/tutorial/single_sphere/sampled/9/second.npz
    python paper_sample_spherical_solution.py \
      -k 9 \
      --fill 0 \
      --sampling-radius 0.090 \
      --config FEM/solutions/tutorial/single_sphere/third.ini \
      --output FEM/solutions/tutorial/single_sphere/sampled/9/third.npz
    
It may take several hours (at least 2.5 GB of free RAM is required).

> Note, that for spherical geometries a dedicated tool
> `paper_sample_spherical_solution.py` is used.

| Parameter  | Description  |
|:------------|:--------------|
| `-k`   | binary logarithm of sample number in each dimansion (which is `2**k + 1`) |
| `--fill` | fill value for points where solution cannot be sampled |
| `--sampling-radius` | radius of the sampled sphere |
| `--config` | the solution metadata |
| `--output` | file for the sampled solution |

# Kernel construction

## Electrode object

An electrode object contains information about electrode spatial location (`.x`, `.y` and `.z` attribute), which is an absolute minimum to be used by kESI (in this case: kCSD with known base function in potential space).  It may also provide additional information about:
- base leadfield (`.base_potential()` method) which enable kCSD for arbitrary base function in CSD space, or
- leadfield correction (`.correction_potential()` method) which enable kESI for setups violating kCSD assumptions,
- base conductivity (`.base_conductivity` attribute) assumed when calculating the leadfield correction.

In [ ]:
import numpy as np

class Electrode(object):
    def __init__(self, filename, decimals_tolerance=None, dx=0):
        """
        Parameters
        ----------
        
        filename : str
            Path to the sampled correction potential.
            
        decimals_tolerance : int
            Precision of coordinate comparison
            in the `.correction_potential()` method.
            
        dx : float
            Integration step used to calculate a regularization
            parameter of the `.base_potential()` method.
        """
        self.filename = filename
        self.decimals_tolerance = decimals_tolerance
        self.dx = dx
        with np.load(filename) as fh:
            self._X = self.round(fh['X'])
            self._Y = self.round(fh['Y'])
            self._Z = self.round(fh['Z'])
            self.x, self.y, self.z = fh['LOCATION']
            self.base_conductivity = fh['BASE_CONDUCTIVITY']

    @property
    def _epsilon(self):
        """
        Regularization parameter of the `.base_potential()` method.
        
        Note
        ----
        
        The 0.15 factor choice has been based on a toy numerical experiment.
        Further, more rigorous experiments are definitely recommended.
        """
        return 0.15 * self.dx
    
    def round(self, A):
        if self.decimals_tolerance is None:
            return A
        return np.round(A, decimals=self.decimals_tolerance)

    def correction_potential(self, X, Y, Z):
        """
        Parameters
        ----------
        X, Y, Z : np.array
            Coordinate matrices with matrix indexing.
            Coordinates are expected to be - respectively -
            from `._X`, `._Y` and `._Z` attributes.
            May be obtained with
            `X, Y, Z = np.meshgrid(..., indexing='ij')`.
        """
        _X, IDX_X, _ = np.intersect1d(self._X, self.round(X[:, 0, 0]), return_indices=True)
        assert len(_X) == np.shape(X)[0]
        _Y, IDX_Y, _ = np.intersect1d(self._Y, self.round(Y[0, :, 0]), return_indices=True)
        assert len(_Y) == np.shape(Y)[1]
        _Z, IDX_Z, _ = np.intersect1d(self._Z, self.round(Z[0, 0, :]), return_indices=True)
        assert len(_Z) == np.shape(Z)[2]

        with np.load(self.filename) as fh:
            return fh['CORRECTION_POTENTIAL'][np.ix_(IDX_X, IDX_Y, IDX_Z)]

    def base_potential(self, X, Y, Z):
        return (0.25 / (np.pi * self.base_conductivity)
                / (self._epsilon
                   + np.sqrt(np.square(X - self.x)
                             + np.square(Y - self.y)
                             + np.square(Z - self.z))))

In [ ]:
electrodes = [Electrode(f'FEM/solutions/tutorial/single_sphere/sampled/9/{name}.npz')
              for name in ['first', 'second', 'third']]

## FRR: Fast Reciprocal Reconstructor 

The `_fast_reciprocal_reconstructor` experimental module contains tools which allow for fast construction of (cross)kernels.  They use discrete Fourier transform (FFT) for high throughput integration.

### Source objects

kESI (and its special case kCSD) reconstructs CSD as a mixture of base functions (in short: _bases_).  FRR tools derive all bases as translation of an appropriate model base (with centroid at the origin of the coordinate system: $[0, 0, 0]$).

While FRR tools operate on base profiles defined as callables accepting vector arguments x, y and z, we can use a convenience kCSD base object (of either `SphericalSplineSourceKCSD` or `GaussianSourceKCSD3D` class) which couple functions in potential and CSD space.  The object is named _source_, as it represents potential (`.potential()` method) generated by certain CSD profile (`.csd()` method) in infinite homogeneous isotropic (kCSD assumptions) medium.

#### Gaussian source

To familiarize with kCSD base function objects, we create an object for CSD given by three dimensional normal distribution centered at $[-5, 20, 100]$ with standard deviation of 10, in a medium of conductivity 0.33 S/m.

In [ ]:
from _common_new import GaussianSourceKCSD3D

import matplotlib.pyplot as plt
import cbf

src = GaussianSourceKCSD3D(-5, 20, 100,
                           standard_deviation=10,
                           conductivity=0.33)

print(f'centroid at ({src.x}m, {src.y}m, {src.z}m)')

As both CSD and potential have spherical symmetry (they depend on the distance to the centroid only), we plot them as functions of the distance:

In [ ]:
_X = src.x + np.linspace(0, 24, 1025)
_Y = src.y + np.linspace(0, 30, 1025)
_Z = src.z + np.linspace(0, 32, 1025)
_R = np.sqrt(np.square(_X - src.x)
             + np.square(_Y - src.y)
             + np.square(_Z - src.z))

_CSD = src.csd(_X, _Y, _Z)
_POTENTIAL = src.potential(_X, _Y, _Z)

plt.figure()
plt.plot(_R, _CSD, color=cbf.SKY_BLUE)

for x in range(10, 50, 10):
    plt.axvline(x, ls=':', color=cbf.BLACK)

plt.xlim(0, 50)
plt.ylabel('CSD [$\\frac{A}{m^3}$]')
plt.xlabel('R [$m$]')

plt.figure()
plt.plot(_R, _POTENTIAL, color=cbf.PURPLE)

for x in range(10, 50, 10):
    plt.axvline(x, ls=':', color=cbf.BLACK)

plt.xlim(0, 50)
plt.ylabel('potential [$V$]')
_ = plt.xlabel('R [$m$]')

It may seem that base function in the CSD space disappears completely in distance greater than 4 standard deviations, but if we plot it in semilogarithmic axes, we can see that its support is much larger (theoretically infinite):

In [ ]:
_X = src.x + np.linspace(0, 240, 1025)
_Y = src.y + np.linspace(0, 300, 1025)
_Z = src.z + np.linspace(0, 320, 1025)
_R = np.sqrt(np.square(_X - src.x)
             + np.square(_Y - src.y)
             + np.square(_Z - src.z))

_CSD = src.csd(_X, _Y, _Z)

plt.figure()
plt.plot(_R, _CSD, color=cbf.SKY_BLUE)

plt.xlim(0, 500)
plt.ylabel('CSD [$\\frac{A}{m^3}$]')
plt.xlabel('R [$m$]')
plt.yscale('log')

Such base functions have certain (luckily minor) drawbacks:
- yield a fuzzy and (theoretically) spatially unlimited CSD reconstructions,
- require sampling the model base globally rather than locally.

It is feasible to mitigate that drawbacks by cropping the function support, which on the other hand introduces an error by design.  Luckily the error is sufferable - for example, cropping the 3D normal distribution to a sphere with radius of 3 standard deviations leads to loss of 2.9% of the distribution (0.81% if cropping to a cube).

But we can do even better and reduce the error to 0 by use of CSD base functions with finite support, like functions of distance from their centroids ($r$; thus spherically symmetric), defined piecewise by polinomials (spline):

$$
\tilde{b}(r) =
\begin{cases}
 p_0(r), 0 \leq r < r_0   \\
 p_1(r), r_0 \leq r < r_1 \\
 \vdots \\
 p_n(r), r_{n-1} \leq r < r_n \\
 0, r_n \le r
\end{cases} ,
$$

where $p_i(x) = \sum_j \alpha_ij x^j$.

We are going to use the `SphericalSplineSourceKCSD` to define a model base with support of radius $R \equiv r_1$, that:
- is constant ($p_0(r) = c$) within radius $r_0 = \frac{1}{3} R$ from centroid,
- is continuous ($p_1(r_0) = c$ and $p_1(r_1) = 0$,
- has continuous derivative ($\dot{p}_1(r_0) = \dot{p}_1(r_1) = 0$, as $p_0$ and $0$ are constant),
- is normalised ($\iiint_{\mathbb{R}^3} \tilde{b}(x, y, z) dv = \int_0^R 4 \pi r^2 \tilde{b}(r) dr = 1$),

thus the coefficients of polynomials are:
$$
\begin{array}{rclcl}
\alpha_{00} & = & c , & & & \\
\alpha_{10} & = & c \, r_2^2\frac{r_2 - 3r_1}{(r_2 - r_1)^3} & = & 0 ,\\
\alpha_{11} & = & c \, 6 \frac{r_1 r_2}{(r_2 - r_1)^3} & = & c \frac{27}{4} R^{-1} ,\\
\alpha_{12} & = & -c \, 3 \frac{r_1 + r_2}{(r_2 - r_1)^3} & = & -c \frac{27}{2} R^{-2} ,\\
\alpha_{13} & = & c \frac{2}{(r_2 - r_1)^3} & = & c \frac{27}{4} R^{-3} ,\\
\end{array}
$$
up to a constant factor $c$.  We do not need to calculate the $c$ factor explicitely, as CSD base functions implemented with `SphericalSplineSourceKCSD` are normalised.  For the sake of simplicity we assume $c = 1$ when passing the coefficients to `SphericalSplineSourceKCSD`:

In [ ]:
from _common_new import SphericalSplineSourceKCSD

def get_model_source(radius, conductivity):
    spline_nodes = [radius / 3, radius]
    spline_polynomial_coefficients = [[1],
                                      [0,
                                       6.75 / radius,
                                       -13.5 / radius ** 2,
                                       6.75 / radius ** 3]]
    return SphericalSplineSourceKCSD(0, 0, 0,
                                     spline_nodes,
                                     spline_polynomial_coefficients,
                                     conductivity)

Here we assume $R = 30$:

In [ ]:
src = get_model_source(30, 0.33)
print(f'centroid at ({src.x}m, {src.y}m, {src.z}m)')
print(f'c = {src.csd(0, 0, 0)}')

_X = np.linspace(0, 40, 1025)

_CSD = src.csd(_X, 0, 0)
_POTENTIAL = src.potential(_X, 0, 0)

plt.figure()
plt.plot(_X, _CSD, color=cbf.SKY_BLUE)

for x in range(10, 40, 10):
    plt.axvline(x, ls=':', color=cbf.BLACK)

plt.xlim(0, 40)
plt.ylabel('CSD [$\\frac{A}{m^3}$]')
plt.xlabel('R [$m$]')

plt.figure()
plt.plot(_X, _POTENTIAL, color=cbf.PURPLE)

for x in range(10, 40, 10):
    plt.axvline(x, ls=':', color=cbf.BLACK)

plt.xlim(0, 40)
plt.ylabel('potential [$V$]')
_ = plt.xlabel('R [$m$]')

We can test whether the CSD base function is normalised by integrating it:

In [ ]:
from scipy.integrate import romb

_k = 8
_X = np.linspace(-30, 30, 2**_k + 1)
_dx = 60 / 2**_k
_SAMPLES = src.csd(_X.reshape(-1, 1, 1),
                   _X.reshape(1, -1, 1),
                   _X.reshape(1, 1, -1))
print(romb(romb(romb(_SAMPLES, dx=_dx), dx=_dx), dx=_dx))
del _SAMPLES

### Convolver object

The convolver is the engine of the FRR tools.  It is used to:
- integrate leadfields weighted by a CSD profile,
- obtain CSD profile of a mixture of base functions.

The convolver operates on three regular 3D grids of coordinates:
- _POT_ grid used for leadfield (_reciprocal potential_) integration,
- _CSD_ grid used for CSD profile calculation,
- _SRC_ grid used for distributing of base function centroids.

The _SRC_ grid is an intersection (in the set operation sense) of the _POT_ and the _CSD_ grids, thus they define the convolver unequivocally.

In [ ]:
from _fast_reciprocal_reconstructor import ckESI_convolver

In [ ]:
_X = electrodes[0]._X
_Y = electrodes[0]._Y
_Z = electrodes[0]._Z


_pot_mesh = [_X, _Y, _Z]
_csd_mesh = [_X, _Y, _Z[_Z >= 0]]

convolver = ckESI_convolver(_pot_mesh, _csd_mesh)

In [ ]:
print(convolver.csd_shape)

In [ ]:
for name in ['POT', 'CSD', 'SRC']:
    print(f'{name} mesh')
    print('  shape:', convolver.shape(name))
    print('  spacing:', convolver.ds(name))

Open 3D meshgrids may be accessed as `.{NAME}_MESH` attributes, where `{NAME}` is the name of the mesh.
Components of each meshgrid may be accessed as `.{NAME}_{C}` attributes, where `{C}` is the name of the coordinate.

### Model source

As convolver will use the Romberg method for integration ($2^k + 1$ quadrature nodes), the integrated CSD profile will be cropped to $2^{k - 1} h$ from centroid in every dimension, where $h$ is the quadrature step size (distance between adjacent _POT_ mesh nodes).  To avoid errors by design, we prefer CSD profiles which support fits in the crop box.  We use the `get_model_source()` function to obtain a pair of model bases which fulfill this condition.

In [ ]:
ROMBERG_K = 6
h_min = min(convolver.ds('POT'))
SRC_R = 2 ** (ROMBERG_K - 1) * h_min
BASE_CONDUCTIVITY = electrodes[0].base_conductivity

model_src = get_model_source(SRC_R, BASE_CONDUCTIVITY)
print(SRC_R)

### Convolver interface

The convolver interface binds the convolver to:
- a CSD profile,
- weights of a quadrature of equally-spaced nodes,
- boolean mask of nodes of the _SRC_ mesh with centroids of the base functions.

We derive quadrature weights by applying Romberg's method to identity matrix ($2^k +1$ one-hot vectors), which yields the effective weights used by the method.

When analytical solution of the kCSD forward problem is used coupled with numeric potential correction, the domain of CSD used for calculation of the potential correction is limited to the support of the leadfield correction.  Thus, close to the support boundary the numeric correction is calculated for different CSD profile than was used for the corrected analytical solution.  To avoid errors, it is advised not to put centroids near the boundary of the support (which is a subset of the _POT_ grid).  It is also advised to limit supports of CSD bases to where current sources are biologically possible (here we include only CSD bases which supports fit in the brain).

In [ ]:
from _fast_reciprocal_reconstructor import ConvolverInterfaceIndexed
from scipy.integrate import romb

ROMBERG_N = 2 ** ROMBERG_K + 1
ROMBERG_WEIGHTS = romb(np.identity(ROMBERG_N), dx=2 ** -ROMBERG_K)
BRAIN_RADIUS = 0.09

SRC_IDX = ((convolver.SRC_Z > 0)
           & (np.sqrt(np.square(convolver.SRC_X)
                      + np.square(convolver.SRC_Y)
                      + np.square(convolver.SRC_Z)) < BRAIN_RADIUS - SRC_R))

In [ ]:
print(SRC_IDX.sum())

In [ ]:
convolver_interface = ConvolverInterfaceIndexed(convolver,
                                                model_src.csd,
                                                ROMBERG_WEIGHTS,
                                                SRC_IDX)

### Kernel constructor and cross-kernel constructor

The kernel constructor is an object which is a collection of callables (methods) facilitating construction of base function images at electrodes ($\Phi$ matrix) and the kernel matrix.  The methods are:
- `.create_base_images_at_electrodes()`  which constructs $\Phi$ from a sequence of electrode objects and a _PAE_ (Potential At Electrode) object responsible for base functions in the potential space,
- `.create_kernel()` which constructs the kernel matrix from $\Phi$.

In [ ]:
from _fast_reciprocal_reconstructor import ckESI_kernel_constructor, ckESI_crosskernel_constructor

In [ ]:
kernel_constructor = ckESI_kernel_constructor()

In [ ]:
CSD_IDX = np.ones(convolver.shape('CSD'),
                  dtype=bool)

The cross-kernel constructor is a callable which - based on the $\Phi$ matrix (and boolean mask of the _CSD_ grid) - constructs the cross-kernel.  We assign it to `kernel_constructor.create_crosskernel` attribute to keep all kernel construction callables in one object.

In [ ]:
kernel_constructor.create_crosskernel = ckESI_crosskernel_constructor(convolver_interface,
                                                                      CSD_IDX)

As all elements of `CSD_IDX` are true, conversion of the reconstructed CSD vector is as simple as its rearrangement to match the _CSD_ grid of `convolver`.  We define an auxilary `to_3D()` function for that purpose.

In [ ]:
def to_3D(CSD):
    return CSD.reshape(convolver.shape('CSD'))

### Potential At Electrode: analytical solution of the kCSD forward problem

In [ ]:
from _fast_reciprocal_reconstructor import PAE_kCSD_Analytical

In [ ]:
pae_kcsd_a = PAE_kCSD_Analytical(convolver_interface,
                                 potential=model_src.potential)

In [ ]:
%%time
PHI_KCSD_ANALYTICAL = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                                          pae_kcsd_a)

In [ ]:
KERNEL_KCSD_ANALYTICAL = kernel_constructor.create_kernel(PHI_KCSD_ANALYTICAL)

In [ ]:
%%time
CROSSKERNEL_KCSD_ANALYTICAL = kernel_constructor.create_crosskernel(PHI_KCSD_ANALYTICAL)

### Potential At Electrode: numerical solution of the kCSD forward problem

In [ ]:
from _fast_reciprocal_reconstructor import PAE_kCSD_Numerical

In [ ]:
pae_kcsd_n = PAE_kCSD_Numerical(convolver_interface)

In [ ]:
%%time
PHI_KCSD_NUMERICAL = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                                         pae_kcsd_n)

In [ ]:
KERNEL_KCSD_NUMERICAL = kernel_constructor.create_kernel(PHI_KCSD_NUMERICAL)

In [ ]:
%%time
CROSSKERNEL_KCSD_NUMERICAL = kernel_constructor.create_crosskernel(PHI_KCSD_NUMERICAL)

### Potential At Electrode: kESI corrected analytical solution of the kCSD forward problem

In [ ]:
from _fast_reciprocal_reconstructor import PAE_kESI_Analytical

In [ ]:
pae_kesi_a = PAE_kESI_Analytical(convolver_interface,
                                 potential=model_src.potential)

In [ ]:
%%time
PHI_KESI_ANALYTICAL = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                                          pae_kesi_a)

In [ ]:
KERNEL_KESI_ANALYTICAL = kernel_constructor.create_kernel(PHI_KESI_ANALYTICAL)

In [ ]:
%%time
CROSSKERNEL_KESI_ANALYTICAL = kernel_constructor.create_crosskernel(PHI_KESI_ANALYTICAL)

### Potential At Electrode: kESI corrected numerical solution of the kCSD forward problem

In [ ]:
from _fast_reciprocal_reconstructor import PAE_kESI_Numerical

In [ ]:
pae_kesi_n = PAE_kESI_Numerical(convolver_interface)

In [ ]:
%%time
PHI_KESI_NUMERICAL = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                                         pae_kesi_n)

In [ ]:
KERNEL_KESI_NUMERICAL = kernel_constructor.create_kernel(PHI_KESI_NUMERICAL)

In [ ]:
%%time
CROSSKERNEL_KESI_NUMERICAL = kernel_constructor.create_crosskernel(PHI_KESI_NUMERICAL)

# Reconstructor

In [ ]:
from kesi._verbose import _CrossKernelReconstructor as Reconstructor
from kesi._engine import _LinearKernelSolver as KernelSolver

In [ ]:
reconstructor_kcsd_a = Reconstructor(KernelSolver(KERNEL_KCSD_ANALYTICAL),
                                     CROSSKERNEL_KCSD_ANALYTICAL)

In [ ]:
reconstructor_kesi_a = Reconstructor(KernelSolver(KERNEL_KESI_ANALYTICAL),
                                     CROSSKERNEL_KESI_ANALYTICAL)

In [ ]:
reconstructor_kcsd_n = Reconstructor(KernelSolver(KERNEL_KCSD_NUMERICAL),
                                     CROSSKERNEL_KCSD_NUMERICAL)

In [ ]:
reconstructor_kesi_n = Reconstructor(KernelSolver(KERNEL_KESI_NUMERICAL),
                                     CROSSKERNEL_KESI_NUMERICAL)

# FEM forward modelling

In [ ]:
import configparser
import dolfin
import scipy.interpolate as si

import FEM.fem_common as fc

We define a FEM forward model. It is a callable, which accepts CSD profile as a callable compatible with `scipy.interpolate.RegularGridInterpolator`.  Note that the model uses a different Dirichlet boundary condition than a forward model for slice geometry.

In [ ]:
class ForwardModel(object):
    GROUNDED_PLATE_AT = -0.088

    def __init__(self, config):
        metadata = fc.MetadataReader(config)
        self.fm = fc.FunctionManager(metadata.getpath('fem', 'mesh'),
                                     metadata.getint('fem', 'degree'),
                                     metadata.get('fem', 'element_type'))
        self.config = configparser.ConfigParser()
        self.config.read(metadata.getpath('model', 'config'))
        
        self.V = self.fm.function_space
        mesh = self.fm.mesh

        n = self.V.dim()
        d = mesh.geometry().dim()

        self.dof_coords = self.V.tabulate_dof_coordinates()
        self.dof_coords.resize((n, d))
        
        self.csd_f = self.fm.function()
        
        self.subdomains = self.fm.load_subdomains()
        self.dx = dolfin.Measure("dx")(subdomain_data=self.subdomains)

    @property
    def CONDUCTIVITY(self):
        for section in self.config.sections():
            if self._is_conductive_volume(section):
                yield (self.config.getint(section, 'volume'),
                       self.config.getfloat(section, 'conductivity'))

    def _is_conductive_volume(self, section):
        return (self.config.has_option(section, 'volume')
                and self.config.has_option(section, 'conductivity')) 
        
    def __call__(self, csd_interpolator):
        self.csd_f.vector()[:] = csd_interpolator(self.dof_coords)
        
        dirichlet_bc_gt = dolfin.DirichletBC(self.V,
                                     dolfin.Constant(0),
                                     (lambda x, on_boundary:
                                      on_boundary and x[2] <= self.GROUNDED_PLATE_AT))
        test = self.fm.test_function()
        trial = self.fm.trial_function()
        potential = self.fm.function()
        
        
        dx = self.dx
        a = sum(dolfin.Constant(c)
                * dolfin.inner(dolfin.grad(trial),
                               dolfin.grad(test))
                * dx(i)
                for i, c
                in self.CONDUCTIVITY)
        L = self.csd_f * test * dx
        
        b = dolfin.assemble(L)
        A = dolfin.assemble(a)
        dirichlet_bc_gt.apply(A, b)
        
        solver = dolfin.KrylovSolver("cg", "ilu")
        solver.parameters["maximum_iterations"] = 10000
        solver.parameters["absolute_tolerance"] = 1E-8
        solver.solve(A, potential.vector(), b)
        
        return potential

In [ ]:
%time fem = ForwardModel('FEM/solutions/tutorial/single_sphere/first.ini')

To test, whether the forward model is compatible with kESI:

1. Take an eigenvector of kESI.
2. Use kESI to reconstruct its CSD profile (~eigensource).
3. Use the forvard model to calculate potential at electrodes for the reconstructed CSD.
4. Plot the measured potentials in eigenvector base.

If the backward and forward models are compatible, the measured potential should be close to the original eigenvector (dot-product close to 1) and far from the others (dot-product close to 0).

In [ ]:
import matplotlib.pyplot as plt
import cbf

def plot_projection(POTENTIAL, BASE):
    plt.stem(np.matmul(POTENTIAL, BASE))
    plt.axhline(0, ls=':', color=cbf.BLACK)
    plt.axhline(1, ls=':', color=cbf.BLACK)

In [ ]:
%%time
_EIGENVALUES, _EIGENVECTORS = np.linalg.eigh(KERNEL_KESI_ANALYTICAL)
_CSD = to_3D(reconstructor_kesi_a(_EIGENVECTORS[:, 1]))
_csd = si.RegularGridInterpolator(
                                  [getattr(convolver, f'CSD_{x}').flatten()
                                   for x in 'XYZ'],
                                  _CSD,
                                  bounds_error=False,
                                  fill_value=0)
_v = fem(_csd)
_V = np.array([_v(_e.x, _e.y, _e.z) for _e in electrodes])

plot_projection([_v(_e.x, _e.y, _e.z) for _e in electrodes],
                _EIGENVECTORS)

# Reconstruction

## Visualisation

For volumetric data visualisation we define an auxilary functions `crude_plot_data()` and `plot_csd_reconstruction()`.

In [ ]:
def crude_plot_data(DATA,
                    x=None,
                    y=None,
                    z=None,
                    grid=None,
                    dpi=30,
                    cmap=cbf.bwr,
                    title=None,
                    amp=None):
    wx, wy, wz = DATA.shape
    
    if grid is None:
        ix, iy, iz = [w // 2 if a is None else a
                      for a, w in zip([x, y, z],
                                      [wx, wy, wz])]
        x, y, z = ix, iy, iz
        
    else:
        x, y, z = [g.mean() if a is None else a
                   for a, g in zip([x, y, z],
                                   grid)]
        ix, iy, iz = [np.searchsorted(g, a)
                      for a, g in zip([x, y, z],
                                   grid)]

    fig = plt.figure(figsize=((wx + wy) / dpi,
                              (wz + wy) / dpi))
    if title is not None:
        fig.suptitle(title)
    gs = plt.GridSpec(2, 2,
                      figure=fig,
                      width_ratios=[wx, wy],
                      height_ratios=[wz, wy])

    ax_xz = fig.add_subplot(gs[0, 0])
    ax_xz.set_aspect('equal')
    ax_xz.set_ylabel('Z')
    ax_xz.set_xlabel('X')

    ax_yx = fig.add_subplot(gs[1, 1])
    ax_yx.set_aspect('equal')
    ax_yx.set_ylabel('X')
    ax_yx.set_xlabel('Y')

    ax_yz = fig.add_subplot(gs[0, 1],
                            sharey=ax_xz,
                            sharex=ax_yx)
    ax_yz.set_aspect('equal')

    cax = fig.add_subplot(gs[1, 0])
    cax.set_visible(False)

    if amp is None:
        amp = abs(DATA).max()

    if grid is None:
        ax_xz.imshow(DATA[:, iy, :].T,
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower')
        ax_yx.imshow(DATA[:, :, iz],
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower')
        im = ax_yz.imshow(DATA[ix, :, :].T,
                          vmin=-amp,
                          vmax=amp,
                          cmap=cmap,
                          origin='lower')
    else:
        ax_xz.imshow(DATA[:, iy, :].T,
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower',
                     extent=(grid[0].min(), grid[0].max(),
                             grid[2].min(), grid[2].max()))
        ax_yx.imshow(DATA[:, :, iz],
                     vmin=-amp,
                     vmax=amp,
                     cmap=cmap,
                     origin='lower',
                     extent=(grid[1].min(), grid[1].max(),
                             grid[0].min(), grid[0].max()))
        im = ax_yz.imshow(DATA[ix, :, :].T,
                          vmin=-amp,
                          vmax=amp,
                          cmap=cmap,
                          origin='lower',
                          extent=(grid[1].min(), grid[1].max(),
                                  grid[2].min(), grid[2].max()))
        
    ax_xz.axvline(x, ls=':', color=cbf.BLACK)
    ax_xz.axhline(z, ls=':', color=cbf.BLACK)

    ax_yx.axvline(y, ls=':', color=cbf.BLACK)
    ax_yx.axhline(x, ls=':', color=cbf.BLACK)

    ax_yz.axvline(y, ls=':', color=cbf.BLACK)
    ax_yz.axhline(z, ls=':', color=cbf.BLACK)
    fig.colorbar(im, ax=cax)

    return (fig, ((ax_xz, ax_yz),
                  (cax, ax_yx)))

In [ ]:
csd_grid = [_x.flatten() for _x in convolver.CSD_MESH]

In [ ]:
def add_spheres(plot, x, y, z,
                sphere_radii=[0.090]):
    def plot_circle(ax, r):
        ax.add_artist(plt.Circle((0, 0), r,
                                facecolor='none',
                                edgecolor=cbf.BLACK,
                                linestyle=':'))

    (fig, ((ax_xz, ax_yz),
           (cax, ax_yx))) = plot
    for r2 in np.square(sphere_radii):
        plot_circle(ax_xz, np.sqrt(r2 - np.square(y)))
        plot_circle(ax_yz, np.sqrt(r2 - np.square(x)))
        plot_circle(ax_yx, np.sqrt(r2 - np.square(z)))

def plot_csd_reconstruction(CSD, x, y, z, title=''):
    ERROR = CSD - GT_CSD
    error_L2 = np.sqrt(np.square(ERROR).sum() / np.square(GT_CSD).sum())
    amp = max(abs(CSD).max(),
              abs(GT_CSD).max(),
              abs(ERROR).max())
    
    for plot in [crude_plot_data(GT_CSD, title='GT CSD',
                                 grid=csd_grid, x=x, y=y, z=z, dpi=35,
                                 amp=amp),
                 crude_plot_data(CSD, title=f'{title} reconstruction',
                                 grid=csd_grid, x=x, y=y, z=z, dpi=35,
                                 amp=amp),
                 crude_plot_data(ERROR, title=f'{title} error (GT normalized L2 norm: {error_L2:.2g})',
                                 grid=csd_grid, x=x, y=y, z=z, dpi=35,
                                 amp=amp)]:
        add_spheres(plot, x, y, z)

## Ground truth CSD and its image at the electrodes

Derive GT CSD as an eigensource of analytical kCSD.

In [ ]:
%%time
EIGENVALUES_KCSD_ANALYTICAL, EIGENVECTORS_KCSD_ANALYTICAL = np.linalg.eigh(KERNEL_KCSD_ANALYTICAL)
EIGENVALUES_KCSD_ANALYTICAL, EIGENVECTORS_KCSD_ANALYTICAL = EIGENVALUES_KCSD_ANALYTICAL[::-1], EIGENVECTORS_KCSD_ANALYTICAL[:, ::-1]

GT_CSD = to_3D(reconstructor_kcsd_a(EIGENVECTORS_KCSD_ANALYTICAL[:, 0]))

In [ ]:
add_spheres(crude_plot_data(GT_CSD,
                            title='GT CSD',
                            grid=csd_grid,
                            x=0.01,
                            y=0,
                            z=0.065,
                            dpi=70),
            x=0.01,
            y=0,
            z=0.065)

Simulate the potential generated by the ground truth CSD profile.

In [ ]:
%%time
_csd = si.RegularGridInterpolator(csd_grid,
                                  GT_CSD,
                                  bounds_error=False,
                                  fill_value=0)
potential = fem(_csd)

We probe the potential in the whole region of interest to visualise it.

In [ ]:
%%time
with np.nditer([convolver.CSD_X[::4, :, :],
                convolver.CSD_Y[:, ::4, :],
                convolver.CSD_Z[:, :, ::4],
                None]) as it:
    for _x, _y, _z, _res in it:
        try:
            _res[...] = potential(_x, _y, _z)
        except RuntimeError:
            _res[...] = np.nan

    V_ROI = np.ma.masked_invalid(it.operands[3])

In [ ]:
add_spheres(crude_plot_data(V_ROI,
                cmap=cbf.PRGn,
                grid=[_x[::4] for _x in csd_grid],
                x=0.01,
                y=0,
                z=0.065,
                dpi=17,
                title='POTENTIAL'),
            x=0.01,
            y=0,
            z=0.065)

Probe the potential at electrodes.

In [ ]:
GT_V = np.array([potential(_e.x, _e.y, _e.z) for _e in electrodes])

In [ ]:
plot_projection(GT_V, EIGENVECTORS_KCSD_ANALYTICAL)

As you can see, the forward model is incompatible with kCSD, as:
- the original eigenvector is significantly overrepresented in the probed potential,
- the other eigenvectors are significant nonzero components of the simulated potential.

We can see that it leads to erratic kCSD reconstruction, which is mostly composed of error:

In [ ]:
CSD_KCSD = to_3D(reconstructor_kcsd_a(GT_V))

In [ ]:
plot_csd_reconstruction(CSD_KCSD, 0.01, 0, 0.065,
                        title='kCSD')

Meanwhile the kESI reconstruction matches the ground truth CSD quite well:

In [ ]:
CSD_KESI = to_3D(reconstructor_kesi_a(GT_V))

In [ ]:
plot_csd_reconstruction(CSD_KESI, 0.01, 0, 0.065,
                        title='kESI')

## Noise

As no measurement is perfect, we introduce 2% noise to see, how kESI may deal with it:

In [ ]:
np.random.seed(42)
NOISE_V = np.random.normal(loc=GT_V, scale=0.02*np.sqrt(np.square(GT_V).mean()))

In [ ]:
CSD_KESI_NOISE = to_3D(reconstructor_kesi_a(NOISE_V))

In [ ]:
plot_csd_reconstruction(CSD_KESI_NOISE, 0.01, 0, 0.065,
                        title='kESI (noise)')

As you can see, the quality of reconstruction decreased significatntly.

## Regularization

We select a regularization parameter which minimizes cross-validation error.  The span of parameters from which it is being selected should cover eigenvalues of the kernel.

In [ ]:
EIGENVALUES_KCSD_ANALYTICAL = np.linalg.eigvalsh(KERNEL_KCSD_ANALYTICAL)[::-1]
EIGENVALUES_KESI_ANALYTICAL = np.linalg.eigvalsh(KERNEL_KESI_ANALYTICAL)[::-1]
EIGENVALUES_KCSD_NUMERICAL = np.linalg.eigvalsh(KERNEL_KCSD_NUMERICAL)[::-1]
EIGENVALUES_KESI_NUMERICAL = np.linalg.eigvalsh(KERNEL_KESI_NUMERICAL)[::-1]

In [ ]:
_kcsd = plt.plot(EIGENVALUES_KCSD_ANALYTICAL, ls=(0, [3, 3]), marker='x', label='kCSD analytical')
_kesi = plt.plot(EIGENVALUES_KESI_ANALYTICAL, ls=(0, [3, 3]), marker='+', label='kESI analytical')
plt.plot(EIGENVALUES_KCSD_NUMERICAL, ls=(0, [1, 1]), marker='+', color=_kcsd[0].get_color(), label='kCSD numerical')
plt.plot(EIGENVALUES_KESI_NUMERICAL, ls=(0, [1, 1]), marker='x', color=_kesi[0].get_color(), label='kESI numerical')

plt.yscale('log')
plt.legend(loc='best')

In [ ]:
REGULARIZATION_PARAMETERS = np.logspace(5, 15, 10 * 10 + 1)

In [ ]:
from _common_new import cv

In [ ]:
%%time
CV_ERRORS_KCSD_ANALYTICAL = cv(reconstructor_kcsd_a, NOISE_V, REGULARIZATION_PARAMETERS)

In [ ]:
%%time
CV_ERRORS_KESI_ANALYTICAL = cv(reconstructor_kesi_a, NOISE_V, REGULARIZATION_PARAMETERS)

In [ ]:
regularization_parameter_kesi_a = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI_ANALYTICAL)]
regularization_parameter_kcsd_a = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD_ANALYTICAL)]

In [ ]:
_kcsd = plt.plot(REGULARIZATION_PARAMETERS,
                 CV_ERRORS_KCSD_ANALYTICAL,
                 label='kCSD analytical')
_kesi = plt.plot(REGULARIZATION_PARAMETERS,
                 CV_ERRORS_KESI_ANALYTICAL,
                 label='kESI analytical')
plt.axvline(regularization_parameter_kcsd_a,
            ls=':',
            color=_kcsd[0].get_color())
plt.axvline(regularization_parameter_kesi_a,
            ls=':',
            color=_kesi[0].get_color())
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')
plt.legend(loc='best')

In [ ]:
CSD_KESI_CV = to_3D(reconstructor_kesi_a(NOISE_V, regularization_parameter_kesi_a))

In [ ]:
plot_csd_reconstruction(CSD_KESI_CV, 0.01, 0, 0.065,
                        title='kESI (regularized)')